<a href="https://colab.research.google.com/github/Ramki13084/tensorflow_sample/blob/main/RestNet50_Sample_Dog_breed_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add 
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


In [ ]:
import shutil, os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.preprocessing import image

In [ ]:
labels = pd.read_csv("C:/Users/91709/dog-breed-identification/labels.csv")
sample = pd.read_csv('C:/Users/91709/dog-breed-identification/sample_submission.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [ ]:
def app_jpg(id):
    return id+".jpg"

In [ ]:
labels['id'] = labels['id'].apply(app_jpg)
labels['id']

0        000bec180eb18c7604dcecc8fe0dba07.jpg
1        001513dfcb2ffafc82cccf4d8bbaba97.jpg
2        001cdf01b096e06d78e9e5112d419397.jpg
3        00214f311d5d2247d5dfe4fe24b2303d.jpg
4        0021f9ceb3235effd7fcde7f7538ed62.jpg
                         ...                 
10217    ffd25009d635cfd16e793503ac5edef0.jpg
10218    ffd3f636f7f379c51ba3648a9ff8254f.jpg
10219    ffe2ca6c940cddfee68fa3cc6c63213f.jpg
10220    ffe5f6d8e2bff356e9482a80a6e29aac.jpg
10221    fff43b07992508bc822f33d8ffd902ae.jpg
Name: id, Length: 10222, dtype: object

In [ ]:
(unique, counts) = np.unique(labels['breed'], return_counts=True)
print(unique,counts)

['affenpinscher' 'afghan_hound' 'african_hunting_dog' 'airedale'
 'american_staffordshire_terrier' 'appenzeller' 'australian_terrier'
 'basenji' 'basset' 'beagle' 'bedlington_terrier' 'bernese_mountain_dog'
 'black-and-tan_coonhound' 'blenheim_spaniel' 'bloodhound' 'bluetick'
 'border_collie' 'border_terrier' 'borzoi' 'boston_bull'
 'bouvier_des_flandres' 'boxer' 'brabancon_griffon' 'briard'
 'brittany_spaniel' 'bull_mastiff' 'cairn' 'cardigan'
 'chesapeake_bay_retriever' 'chihuahua' 'chow' 'clumber' 'cocker_spaniel'
 'collie' 'curly-coated_retriever' 'dandie_dinmont' 'dhole' 'dingo'
 'doberman' 'english_foxhound' 'english_setter' 'english_springer'
 'entlebucher' 'eskimo_dog' 'flat-coated_retriever' 'french_bulldog'
 'german_shepherd' 'german_short-haired_pointer' 'giant_schnauzer'
 'golden_retriever' 'gordon_setter' 'great_dane' 'great_pyrenees'
 'greater_swiss_mountain_dog' 'groenendael' 'ibizan_hound' 'irish_setter'
 'irish_terrier' 'irish_water_spaniel' 'irish_wolfhound'
 'italian

In [ ]:
desired_breeds = ['beagle', 'chihuahua', 'doberman','french_bulldog', 'golden_retriever', 'malamute', 'pug', 'saint_bernard', 'scottish_deerhound','tibetan_mastiff']
labels1 = labels[labels['breed'].isin(desired_breeds)]
(unique, counts) = np.unique(labels1['breed'], return_counts=True)
print(unique,counts)
labels1.count
os.chdir('C:/Users/91709/dog-breed-identification/')
print(os.getcwd())


['beagle' 'chihuahua' 'doberman' 'french_bulldog' 'golden_retriever'
 'malamute' 'pug' 'saint_bernard' 'scottish_deerhound' 'tibetan_mastiff'] [105  71  74  70  67  81  94  84 126  69]
C:\Users\91709\dog-breed-identification


In [ ]:
directory = "sample_train"
  

parent_dir = "C:/Users/91709/dog-breed-identification/"
  
# Path
path = os.path.join(parent_dir, directory)
  
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir(path)

FileExistsError: ignored

In [ ]:
labels1.to_csv('labels1.csv')

In [ ]:
os.getcwd()

'C:\\Users\\91709\\dog-breed-identification'

In [ ]:
for f in labels1['id']:
    shutil.copy('C:/Users/91709/dog-breed-identification/train/' + f, 'C:/Users/91709/dog-breed-identification/sample_train/')

In [ ]:
def res_identity(x,filters):

      x_skip =x
      f1,f2 = filters

        #first block 
      x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
      x = BatchNormalization()(x)
      x = Activation(activations.relu)(x)

      #second block # bottleneck (but size kept same with padding)
      x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
      x = BatchNormalization()(x)
      x = Activation(activations.relu)(x)

      # third block activation used after adding the input
      x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
      x = BatchNormalization()(x)
      # x = Activation(activations.relu)(x)

      # add the input 
      x = Add()([x, x_skip])
      x = Activation(activations.relu)(x)

      return x

In [ ]:
def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [ ]:
def resnet50():

  input_im = Input(shape=(train_im.shape[1], train_im.shape[2], train_im.shape[3])) # cifar 10 images size
  x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_conv(x, s=2, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_conv(x, s=2, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_conv(x, s=2, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

  x = AveragePooling2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(len(class_types), activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model